# Поиск выигрышной стратегии в карточной игре (вероятностное программирование)

### Условие задачи

#### Дано: 
- Колода 52 карты, повернуты рубашкой вверх. 
- Одна половина карт - красная, вторая - черная. 
- Карты переворачивают одну за другой. 
- Если последняя перевернутая карта окажется красной - выигрыш +1.  
- Если последняя перевернутая карта окажется черной - проигрыш -1.
- Игра может окончиться в любой момент. При желании игроков. 
- Можно играть сколько угодно раз. 
- Каждый раз колода перетасовывается. 
- Расчет после каждого раунда
#### To-do:
- 1) Вычислить вероятность событий при помощи анализа пространства элементарных исходов.
- 2) Отобразить вероятность событий для ряда значений интервалов.
- 3) Симулировать случайные процессы, такие как подбрасывание монеты, перетасовка карт при помощи Python. 
- 4) Оценить уверенность в решениях, полученных в результате симуляций, используя анализ доверительных интервалов.

## 1.1 Анализ пространсва элементарных исходов 

#### Подбрасывание честной монеты как пространство элементарных исходов.
Создание вероятностного пространства

In [3]:
# Создание пространства Орлы / решки
sample_space = {'Heads', 'Tails'}
# Heads как и Tails имеют частоту 1/2
probability_heads = 1 / len(sample_space)
print(f'Probability of choosing heads is {probability_heads}')
# Мы описали вероятность события. Событие - это подмножество элементов в sample_space, 
# которые удовлетворяют условию события. Условие события - простая логическая функция
# Функция принимает на вход элемент из sample_space и возвращает True, только
# если этот элемент удовлетворяет требованиям условий.

Probability of choosing heads is 0.5


Определение условий

In [7]:
# Определим условия: "Выпад орла или решки", "Ни одно из них"
def is_heads_or_tails(outcome): return outcome in {'Heads', 'Tails'}
def is_neither(outcome): return not is_heads_or_tails(outcome)
# Определение условий для базовых событий: "Выпад орла", "Выпад решки"
def is_heads(outcome): return outcome == "Heads"
def is_tails(outcome): return outcome == "Tails"

Создание функции выявления события. На вход получает условие события. Перебрав полученное пространство, эта функция возвращает ножество исходов, в котором event_condition(outcome) равен True

In [13]:
# Создаем функцию выявления события
def get_matching_event(event_condition, sample_space):
    return set([outcome for outcome in sample_space
                if event_condition(outcome)])

Выполняем функцию для всех четырех условий

In [20]:
# Создаем список с условиями 
event_conditions = [is_heads_or_tails, is_heads, is_tails, is_neither]

for event_condition in event_conditions: 
    print(f"Event Condition: {event_condition.__name__}")
    event = get_matching_event(event_condition, sample_space)
    print(f'Event: {event}\n')


Event Condition: is_heads_or_tails
Event: {'Heads', 'Tails'}

Event Condition: is_heads
Event: {'Heads'}

Event Condition: is_tails
Event: {'Tails'}

Event Condition: is_neither
Event: set()



Вычисление вероятности события. Вероятность многоэлементного события для честной монеты равна размеру этого события, разделенному на размер пространства элементарных исходов. 

In [24]:
# Создание функции compute_probability (извлекает событие, вычисляет его вероятность)
def compute_probability(event_condition, generic_sample_space):
    event = get_matching_event(event_condition, generic_sample_space)
    return len(event) / len(generic_sample_space)
# Цикл для выполнения функции
for event_condition in event_conditions: 
    prob = compute_probability(event_condition, sample_space)
    name = event_condition.__name__
    print(f"Probability of event arising from '{name}' is {prob}")

Probability of event arising from 'is_heads_or_tails' is 1.0
Probability of event arising from 'is_heads' is 0.5
Probability of event arising from 'is_tails' is 0.5
Probability of event arising from 'is_neither' is 0.0


Получен диапазон вероятностей событий (0-1). Нижняя и верхняя границы. 

#### 1.1.1. Анализ несимметричной монеты

Вероятность выпадения орлом в четыре раза выше, чем выпадение решкой. Создаем словарь с весами. 

In [25]:
# Создаем пространство взвешанных весов
weighted_sample_space = {'Heads': 4, "Tails": 1}

In [37]:
# Проверим размер пространства взвешанных весов 
sample_space_size = sum(weighted_sample_space.values())
assert sample_space_size == 5
# Суммирование всех веслов дает размер события
event = get_matching_event(is_heads_or_tails, weighted_sample_space)
event_size = sum(weighted_sample_space[outcome] for outcome in event) 
assert event_size == 5

In [44]:
# Определяем обощенную функцию вычисления вероятности события
def compute_event_probability(event_condition, generic_sample_space):
    event = get_matching_event(event_condition, generic_sample_space)
    if type(generic_sample_space) == type(set()):
        return len(event) / len(generic_sample_space)
    event_size = sum(generic_sample_space[outcome]
                    for outcome in event)
    return event_size / sum(generic_sample_space. values())

In [45]:
# Цикл для вывода вероятностей событий
for event_condition in event_conditions: 
    prob = compute_event_probability(event_condition, weighted_sample_space)
    name = event_condition.__name__
    print(f"Probability of event arising from '{name}' is {prob}")

Probability of event arising from 'is_heads_or_tails' is 1.0
Probability of event arising from 'is_heads' is 0.8
Probability of event arising from 'is_tails' is 0.2
Probability of event arising from 'is_neither' is 0.0


Применим инструмент к более сложным случаям. 

## 1.2 Вычисление сложных вероятностей

Будем использовать compute_event_probability

#### Анализ семьи с четырьмя детьми